In [1]:
from functions import *

# Committing to scripts via TapTweak

The following material adapted from the Bitcoin Optech [Schnorr Taproot workshop](https://bitcoinops.org/en/schorr-taproot-workshop/).

* Part 1: Tweaking the public key; commitment schemes with tweaks
* Part 2: Spending a (tweaked) taproot output along the key path
* Part 3 (Case Study): contract commitments

The linear property of BIP340 means that we can encode a commitment into a public key, and then reveal that commitment when signing with the private key. We do that by tweaking the private key with the commitment, and using the associated tweaked pubkey. When signing, we can reveal that the original keys were tweaked by the commitment.

In part 1, we'll learn about how private/public key pairs can be tweaked, and how we can use that to create a secure commitment scheme. In part 2, we'll create a segwit v1 output and spend it along the key path, using a tweaked private and public key. Part 3 of this chapter is a case study, showing how pay-to-contract with tweaked keys can be used instead of OP_RETURN outputs to create timestamped commitments.

In [30]:
# Generate a key pair
privkey = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000001")
pubkey = tr.pubkey_gen(privkey)
print(f"Private key: {privkey.hex()}\nPublic key: {pubkey.hex()}\n")

msg = sha256(b'msg')
aux_rand = bytes(32) # auxiliary random data
sig = tr.schnorr_sign(msg, privkey, aux_rand)
print(f"Signature: {sig.hex()}\n")

assert(tr.schnorr_verify(msg, pubkey, sig))
print("Success!")

Private key: 0000000000000000000000000000000000000000000000000000000000000001
Public key: 79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798

Signature: 4791b03f4381ddd5f22847d1c736e64d1d8d02f90af83a1373bb6392aae8d132660cc589d47e13a40c6bce2a408887d56f7ecac3895b9a9e5d75fe78b6f2478e

Success!



## Part 1: Tweaking the public key

Instead of using our original public key as the witness program, we use a tweaked public key.

* `[01] [32B Tweaked Public Key]`

Tweaking a public key means to alter it with a value (the tweak) such that it remains spendable with knowledge of the original private key and tweak.

* `x` / `P`: Original private key / public key pair
* `t` / `T`: Tweak value / tweak point
* Output script: `[01] [P + T]` = `[01] [xG + tG]`
* Spendable by the tweaked private key: `x + t`

An observer cannot distinguish between a tweaked and untweaked public key.

#### Example: Signing with a tweaked keypair

In this example, we generate a key pair as before, and then tweak both the private key and public key. We then sign with the tweaked private key and verify that the signature is valid.

A _tweak_ is positive scalar value `t` where `0 < t < SECP256K1_ORDER`. There is an associated tweak point `T` such that `T = t*G`.

The private key is tweaked by the tweak scalar: `x' = x + t` and the public key is tweaked by the tweak point: `P' = P + T`.

The tweaked private key `x'` can be used to produce a valid signature for the tweaked pubkey `P'`.

In [32]:
# Using the same private/public keypair as before
print(f"Private key: {privkey.hex()}\nPublic key: {pubkey.hex()}\n")

# Let's set a random tweak scalar 0 < t < SECP256K1_ORDER and derive its associated tweak point
tweak_val = bytes.fromhex("73fbb916db668bf8d6be37c171403793a890f28a10de7a3da40afe12ef5a4494")

# Generate the tweak point in the same way we'd generate a pubkey from a private key
tweak_point = tr.pubkey_gen(tweak_val)
print(f"Tweak scalar: {tweak_val.hex()}\nTweak point: {tweak_point.hex()}\n")

# Derive the tweaked private key and public key
privkey_int = int.from_bytes(privkey, byteorder="big")
tweak_int = int.from_bytes(tweak_val, byteorder="big")
privkey_tweaked_int = (privkey_int + tweak_int) % tr.SECP256K1_ORDER

# Convert the tweaked private key back to bytes
privkey_tweaked = privkey_tweaked_int.to_bytes(32, "big")

# Generate the corresponding tweaked public key in the usual way
pubkey_tweaked = tr.pubkey_gen(privkey_tweaked)

# Sign the message with tweaked key pair and verify the signature
msg = sha256(b'msg')
aux = bytes(32)
sig = tr.schnorr_sign(msg, privkey_tweaked, aux_rand)

assert(tr.schnorr_verify(msg, pubkey_tweaked, sig))
print("Success!")

Private key: 0000000000000000000000000000000000000000000000000000000000000001
Public key: 79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798

Tweak scalar: 73fbb916db668bf8d6be37c171403793a890f28a10de7a3da40afe12ef5a4494
Tweak point: 2526c792e88c0130779ca676e56f1c47e197b53307ecfb35e526238b1bfee43d

Success!


In [28]:
def taproot_tweak_seckey(seckey0, h):
    seckey0 = tr.int_from_bytes(seckey0)
    P = tr.point_mul(tr.G, seckey0)
    seckey = seckey0 if tr.has_even_y(P) else tr.SECP256K1_ORDER - seckey0
    t = tr.int_from_bytes(tr.tagged_hash("TapTweak", tr.bytes_from_int(tr.x(P)) + h))
    if t >= tr.SECP256K1_ORDER:
        raise ValueError
    return tr.bytes_from_int((seckey + t) % tr.SECP256K1_ORDER)

In [29]:
taproot_tweak_seckey(privkey, tweak_val).hex()

'3ff3139fe26f473bc53679984aac274aca09f83bcf6406c1d2df5913b1bb58ab'